In [34]:
prompt = """
You are Luna, a support agent for NC IDEA. Answer questions based on the provided context. The context is from NC IDEA and the users are people trying to learn about or apply to the SEED grant. Don't mention you're an AI unless you're asked. Be polite, helpful, and friendly.

TASK DESCRIPTION:
Respond to the query using the given context.
- Provide short, easily understood answers.
- Use a conversational and friendly style.
- Do not reiterate the question.

If the question is not covered in the context, answer, "I'm sorry, I don't have that information." unless it is basic information that can be inferred from the context.

HARD REQUIREMENTS:
- If part of the query is covered in the documents but the other part isn't, answer the available parts of the question and let the user know what part you don't have information about.
- Keep the answer within 300 tokens.
- Do not instruct users to contact support or other channels.
- Avoid implying you can perform actions.
- Quote what you are using to answer the question.
- Answer the question in the query. Here is the query: "{question}"
- Base answers strictly on the context. Here is the context: ""
{context}
""
"""

In [35]:
from langchain_community.chat_models import ChatOllama
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
from pydantic import BaseModel, Field

embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
llm = ChatOllama(model="llama3.1", device="cuda", temperature=0)

class AgentState(TypedDict):
    results: List[str]
    response: str

class Agent:
    def __init__(self, query="", vectorstore=vectorstore, model=llm):
        graph = StateGraph(AgentState)
        node1 = "search_results"
        node2 = "llm_response"

        graph.add_node(node1, self.sim_search)
        graph.add_node(node2, self.response)

        graph.add_edge(node1, node2)
        graph.add_edge(node2, END)
        
        graph.set_entry_point(node1)
        
        self.graph = graph.compile()
        self.vectorstore = vectorstore
        self.query = query
        self.model = model

    def invoke(self, query):
        self.query = query
        return self.graph.invoke({'results': [], 'response': ""})

    def sim_search(self, state: AgentState):
        search_results = self.vectorstore.similarity_search(self.query, k=2)
        results = [result.page_content for result in search_results]
        state['results'] = results
        return state

    def response(self, state: AgentState):
        context = "\n".join(state['results'])
        formatted_prompt = prompt.format(question=self.query, context=context)
        state['response'] = self.model.invoke(formatted_prompt)
        return state


In [ ]:
from IPython.display import Image

abot = Agent()
Image(abot.graph.get_graph().draw_png())

In [21]:
llm = ChatOllama(model="llama3.2", device="cuda", temperature=0)
abot = Agent(model=llm)
result = abot.invoke("What is the eligibility criteria for the NC IDEA SEED grant?")

In [22]:
abot2 = Agent()
result2 = abot2.invoke("What is the eligibility criteria for the NC IDEA SEED grant?")

In [23]:
print(result['response'].content)
print(result['results'])

To be eligible for the NC IDEA SEED grant, your company must meet certain criteria. According to the eligibility requirements, businesses that are illegal in NC, even if legal in other states or countries, are not eligible.

Additionally, companies in industries with high capital requirements and a long runway to commercialization, such as pharmaceutical companies, are also not eligible.

You can check the specific list of ineligible company types on the application. If you're unsure about your company's eligibility, you can review the industry and company type section to see if it matches any of the ineligible categories.

It's also important to note that NC IDEA SEED grants are specifically targeted at companies headquartered in and conducting a majority of corporate operations in North Carolina.
['THIS IS A TEMPLATE ONLY AND NOT THE NC IDEA APPLICATION.   \nAPPLICATIONS MUST BE COMPLETED AND SUBMITTED ONLINE VIA ncidea.my.site.com\n\nNC IDEA SEED Grant Application \n\nFall 2024\n\nN

In [24]:
print(result2['response'].content)
print(result2['results'])

According to the NC IDEA SEED Grant Application, the eligibility criteria for the NC IDEA SEED grant can be found on page 2 under "SEED Eligibility Criteria". 

The SEED Eligibility Criteria states that companies are not eligible if they fall into one of the following categories:

* Businesses that are illegal in NC
* Companies in industries with high capital requirements and a long runway to commercialization, such as pharmaceutical companies
* Consulting practices or agencies
* Distributors
* Event venue and management companies
* Franchises
* Local, service-oriented small businesses (i.e. restaurants, retail stores, catering, cleaning companies)
* Not-for-profits
* Resellers
* Television/film production companies
* Traditional publishing companies
* Traditional staffing agencies 
* Training companies
* Trucking, hauling, or freight companies
* Other “lifestyle” companies

Additionally, the company must have its headquarters in NC if not all founders reside in NC.
['THIS IS A TEMPLAT

In [ ]:
llm = ChatOllama(model="llama3.2", device="cuda", temperature=0)
abot = Agent(model=llm)
result = abot.invoke("I represent a moving company. Am I allowed to apply?")

In [ ]:
abot2 = Agent()
result2 = abot2.invoke("I represent a moving company. Am I allowed to apply?")

In [38]:
print(result['response'].content)
print(result['results'])

Hello! I'd be happy to help you with your SEED grant application. You represent a moving company, but according to our eligibility criteria, we only consider companies that meet specific industry and company type requirements.

Unfortunately, it appears that the moving industry is not currently eligible for an NC IDEA program or grant. We focus on supporting startups in other industries, such as technology, healthcare, and advanced manufacturing.

If you're interested in exploring alternative funding options or resources, I'd be happy to provide more information on our available programs and services.
["Questions with picklist or date answer choices.\n\nELIGIBILITY: INDUSTRY AND COMPANY TYPE\n\nCOMPANY STRUCTURE\n\nINCORPORATION DATE (if applicable)\n\nCOMPANY LOCATION: NORTH CAROLINA COUNTY \n\nBUSINESS CATEGORY\n\nBUSINESS SUB-CATEGORY\n\nTARGET MARKET \n\nUNIVERSITY AFFILIATION\n\nADVISORS: List any formal and informal business or technical advisors (and their organizations, if appl

In [49]:
print(result2['response'].content)
print(result2['results'])

I'd be happy to help! According to the context, it seems that eligibility for the SEED grant is based on various factors such as industry and company type, incorporation date, location, business category, target market, university affiliation, advisors, supporting communities, previous applicant status, and more.

Regarding your question, "Am I allowed to apply?" - I see that there's a section called "PREVIOUS APPLICANT" which asks if the company has previously applied to an NC IDEA program(s) and/or grant(s). If the answer is yes, then you'll need to provide additional information. However, it doesn't explicitly state whether a moving company can apply or not.

But, I do see that there's no specific restriction mentioned for companies in the moving industry. In fact, the eligibility criteria seem to be quite broad and focused on the company's location, business category, target market, and other factors rather than specific industries.

So, based on this information, it appears that a

In [50]:
result2['results'][1]

"University Name\n\nGender\n\nEthnicity\n\nVeteran status\n\nTEAM: STAFF (NON-FOUNDER) TAB \n\n\nIf your startup has paid or unpaid non-founder staff, please include the following information for each person. \n\nName\n\nTitle\n\nLinkedIn profile URL\n\nInclude a BRIEF description of how this person’s background relates to your company.\n\nFull-time or part-time status at your company\n\nIf a person is part-time, you will be asked to briefly describe what milestone(s) would need to be realized for this person to go full-time, if that is their intent.\n\nBUSINESS DETAILS TAB\n\nVIDEO URL (OPTIONAL): You are welcome to submit a video that you feel would complement or enhance your application. Submitting a video is optional but encouraged. The video could include an elevator pitch, product demo (highly encouraged for software companies), customer story/validation or other type of pitch. Videos should be submitted via a link to the video on YouTube, Vimeo, etc. \n\nWe cannot accept separat